In [4]:
import awkward as ak
import uproot
import hist as hs
import numpy as np

from coffea.nanoevents import NanoEventsFactory, BaseSchema, NanoAODSchema, TreeMakerSchema, PHYSLITESchema
from coffea.nanoevents.methods import vector

In [5]:
signaldomain     = 'root://cmsxrootd.fnal.gov//store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_Fall18/v2/v3/normalized/BToKPhi_MuonGenFilter_mPhi1p0_ctau1000_1pb_weighted.root'
signalfile = uproot.open(signaldomain)

backgrounddomain = 'root://cmsxrootd.fnal.gov//store/user/christiw/displacedJetMuonAnalyzer/csc/V1p171/Data2018_UL/v3/normalized/ParkingBPH4_Run2018A_goodLumi.root'
backgroundfile = uproot.open(backgrounddomain)

In [6]:
signalevents = NanoEventsFactory.from_root(
    signalfile,
    treepath='MuonSystem',
    #entry_stop=10,
    #metadata={"dataset": "llp"},
    schemaclass=BaseSchema,
).events()


backgroundevents = NanoEventsFactory.from_root(
    backgroundfile,
    treepath='MuonSystem',
    #entry_stop=10,
    #metadata={"dataset": "llp"},
    schemaclass=BaseSchema,
).events()

In [4]:
[f'{field}' for field in signalevents.fields if field.startswith('gLLP_')]

['gLLP_eta',
 'gLLP_phi',
 'gLLP_csc',
 'gLLP_dt',
 'gLLP_beta',
 'gLLP_e',
 'gLLP_pt',
 'gLLP_ctau',
 'gLLP_decay_vertex_r',
 'gLLP_decay_vertex_x',
 'gLLP_decay_vertex_y',
 'gLLP_decay_vertex_z']

In [51]:
events = signalevents
gLLPs = ak.zip(
    {
        'pt': events.gLLP_pt,
        'phi': events.gLLP_phi,
        'eta': events.gLLP_eta,
        'E': events.gLLP_e,
    },
    with_name = 'PtEtaPhiMLorentzVector',
    behavior = vector.behavior,                 
  )

clusts = ak.zip(
    {
        'pt': ak.zeros_like(events.cscRechitClusterPhi),
        'phi': events.cscRechitClusterPhi,
        'eta': events.cscRechitClusterEta,
        'E': ak.zeros_like(events.cscRechitClusterPhi),
    },
    with_name = 'PtEtaPhiMLorentzVector',
    behavior = vector.behavior,                 
  )

cls_llp_pairs = ak.cartesian({
    "cls": clusts, 
    "llp": gLLPs
}, nested = True)      

In [52]:
for i, thing in enumerate(cls_llp_pairs):
    if len(thing) > 2:
        print(i, thing)

3758 [[{cls: {pt: 0, phi: -0.123, eta: -2.17, E: 0}, ... phi: 1, eta: -0.296, E: 11.1}}]]
19880 [[{cls: {pt: 0, phi: 0.808, eta: 2.15, E: 0}, ... phi: 2.46, eta: -1.19, E: 8.19}}]]
31643 [[{cls: {pt: 0, phi: 3.03, eta: -2.21, E: 0}, ... eta: -1.31, E: 13.2}}]]
36723 [[{cls: {pt: 0, phi: 2.41, eta: -1.52, E: 0}, ... phi: 2.35, eta: -1.47, E: 5.78}}]]
42071 [[{cls: {pt: 0, phi: 0.119, eta: -2.12, E: 0}, ... phi: 1.98, eta: 1.43, E: 5.57}}]]
49002 [[{cls: {pt: 0, phi: -1.99, eta: -1.5, E: 0}, ... phi: 0.689, eta: -1.28, E: 16.5}}]]
58202 [[{cls: {pt: 0, phi: -0.288, eta: -2.11, E: 0}, ... phi: 2.49, eta: -5.55, E: 381}}]]
71349 [[{cls: {pt: 0, phi: 2.98, eta: 2.09, E: 0}, ... phi: 3, eta: 2.15, E: 4.74}}]]
93105 [[{cls: {pt: 0, phi: -1.05, eta: -2.03, E: 0}, ... eta: -0.972, E: 2.74}}]]
128389 [[{cls: {pt: 0, phi: -2.83, eta: 2.04, E: 0}, ... phi: 0.446, eta: 1.61, E: 20}}]]
149775 [[{cls: {pt: 0, phi: 1.33, eta: 2.23, E: 0}, ... phi: -1.44, eta: 1.86, E: 15.7}}]]
169235 [[{cls: {pt: 0, p

KeyboardInterrupt: 

In [36]:
events[3758].cscRechitClusterEta
#events[3758].gLLP_eta

<Array [-2.17, 2.07, 2.21] type='3 * float32[parameters={"__doc__": "cscRechitCl...'>

<Array [[{cls: {pt: 0, ... E: 11.1}}]] type='3 * 1 * {"cls": PtEtaPhiMLorentzVec...'>